In [1]:
# Prepare the DataSet
import os
import cv2
import numpy as np 

def ShowImgLab(Img_Index):
    
    print(true_labels[Img_Index])
    cv2.imshow('Img', img_arr[Img_Index])
    cv2.waitKey()
    cv2.destroyAllWindows()

def shuffle_in_unison(a, b):
    
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)
    

IMG_WIDTH = 200
IMG_HEIGHT = 200
CLOSED_COUNT = len(os.listdir('.\closed_formated'))
OPEN_COUNT = len(os.listdir('.\open_formated'))
DEV_SIZE = 300
TEST_SIZE = 100
TRAIN_SIZE = CLOSED_COUNT + OPEN_COUNT - DEV_SIZE - TEST_SIZE
print('Closed count: '+ str(CLOSED_COUNT))
print('Open count: '+ str(OPEN_COUNT))
print('Train count: '+ str(TRAIN_SIZE))

img_arr = np.empty([CLOSED_COUNT + OPEN_COUNT, 1, IMG_WIDTH, IMG_HEIGHT], np.uint8)
true_labels = np.empty(CLOSED_COUNT + OPEN_COUNT, np.uint8)

# import Closed eyes, one of the classes
true_labels[0:CLOSED_COUNT] = 1
for dirname, dirnames, filenames in os.walk('.\closed_formated'):
    
    for (i, filename) in enumerate(filenames):
        image = cv2.imread('.\closed_formated\\'+filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        img_arr[i] = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_AREA)
        
# import Open eyes, one of the classes
true_labels[CLOSED_COUNT:CLOSED_COUNT + OPEN_COUNT] = 0
for dirname, dirnames, filenames in os.walk('.\open_formated'):
    
    for (i, filename) in enumerate(filenames):
        image = cv2.imread('.\open_formated\\'+filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Shift after closed 
        img_arr[i+CLOSED_COUNT] = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_AREA)

shuffle_in_unison(img_arr, true_labels)

X_train = img_arr[0:TRAIN_SIZE]
Y_train = true_labels[0:TRAIN_SIZE]

X_dev = img_arr[TRAIN_SIZE:TRAIN_SIZE+DEV_SIZE]
Y_dev = true_labels[TRAIN_SIZE:TRAIN_SIZE+DEV_SIZE]

X_test = img_arr[TRAIN_SIZE+DEV_SIZE:TRAIN_SIZE+DEV_SIZE+TEST_SIZE]
Y_test = true_labels[TRAIN_SIZE+DEV_SIZE:TRAIN_SIZE+DEV_SIZE+TEST_SIZE]

Closed count: 1149
Open count: 810
Train count: 1559


In [ ]:
# Check the DataSet
for index in range(60,80):
    
    print(Y_train[index])
    cv2.imshow('Img', X_train[index,0])
    cv2.waitKey()
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cv2.destroyAllWindows()

In [18]:
def extract_eye(img):
    
    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    
    eye_cascade = eye_cascade.detectMultiScale(img)
    
    for (x,y,w,h) in eye_cascade:
        #img = cv2.rectangle(img, (x,y),(x+w,y+h),(255,0,0),2)
        
        img = img[y:y+h,x:x+w]
        break
        
    img = cv2.resize(img, (30, 30), interpolation = cv2.INTER_AREA)

    return img

In [ ]:
for index in range(430,450):
    cv2.imshow('Img', X_train[index,0])
    cv2.waitKey()
    cv2.imshow('Img', extract_eye(X_train[index,0]))
    cv2.waitKey()
    cv2.destroyAllWindows()

In [3]:
X_eye_train = np.empty([TRAIN_SIZE, 1, 30, 30], np.uint8)
X_eye_dev = np.empty([DEV_SIZE, 1, 30, 30], np.uint8)

for (i, i_train) in enumerate(X_train):
    X_eye_train[i] = extract_eye(i_train[0])
    
for (i, i_dev) in enumerate(X_dev):
    X_eye_dev[i] = extract_eye(i_dev[0])

In [7]:
# Check the DataSet
for index in range(80,100):
    
    print(Y_dev[index])
    cv2.imshow('Img', X_dev[index,0])
    cv2.waitKey()
    cv2.imshow('Img', X_eye_dev[index,0])
    cv2.waitKey()
        
cv2.destroyAllWindows()

0
0
1
0
0
1
0
1
1
1
1
0
0
0
0
1
0
1
0
0


In [10]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_first')

def OpenEyeDetect(input_shape):
    """
    Implementation of the OpenEyeDetect.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)
    
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    #X = Conv2D(32, (7, 7), strides=(1, 1), name='conv0')(X)
    #X = BatchNormalization(axis=3, name='bn0')(X)
    #X = Activation('relu')(X)
    # MAXPOOL
    #X = MaxPooling2D((2, 2), name='max_pool0')(X)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (5, 5), strides=(1, 1), name='conv1')(X)
    X = BatchNormalization(axis=3, name='bn1')(X)
    X = Activation('relu')(X)
    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (5, 5), strides=(1, 1), name='conv2')(X)
    X = BatchNormalization(axis=3, name='bn2')(X)
    X = Activation('relu')(X)
    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool2')(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dropout(0.5)(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs=X_input, outputs=X, name='OpenEyeDetect')

    return model

In [11]:
ins_closedEyesModel = OpenEyeDetect(X_eye_train.shape[1:])

In [12]:
ins_closedEyesModel.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [14]:
ins_closedEyesModel.fit(X_eye_train, Y_train, epochs=15, batch_size=32)

Epoch 1/15
1559/1559 [==============================] - 1s - loss: 0.2478 - acc: 0.8993     
Epoch 2/15
1559/1559 [==============================] - 0s - loss: 0.2218 - acc: 0.9083     
Epoch 3/15
1559/1559 [==============================] - 1s - loss: 0.2209 - acc: 0.9121     
Epoch 4/15
1559/1559 [==============================] - 1s - loss: 0.2117 - acc: 0.9243     
Epoch 5/15
1559/1559 [==============================] - 1s - loss: 0.2224 - acc: 0.9153     
Epoch 6/15
1559/1559 [==============================] - 1s - loss: 0.1899 - acc: 0.9262     
Epoch 7/15
1559/1559 [==============================] - 1s - loss: 0.1720 - acc: 0.9359     
Epoch 8/15
1559/1559 [==============================] - 1s - loss: 0.1911 - acc: 0.9269     
Epoch 9/15
1559/1559 [==============================] - 1s - loss: 0.1688 - acc: 0.9352     
Epoch 10/15
1559/1559 [==============================] - 1s - loss: 0.1502 - acc: 0.9468     
Epoch 11/15
1559/1559 [==============================] - 1s - loss: 0

In [15]:
preds = ins_closedEyesModel.evaluate(X_eye_dev, Y_dev, batch_size=32, verbose=1, sample_weight=None)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

300/300 [==============================] - 0s     

Loss = 0.229616438746
Test Accuracy = 0.920000000795


In [16]:
for index in range(60, 80):
    print(ins_closedEyesModel.predict(X_eye_dev[index:index+1]))
    cv2.imshow('Img', X_dev[index,0])
    cv2.waitKey()
    cv2.destroyAllWindows()

[[ 0.99639624]]
[[ 0.00150032]]
[[ 0.99992836]]
[[ 0.89635199]]
[[ 0.99999475]]
[[ 0.22430634]]
[[ 0.47727466]]
[[ 0.95485944]]
[[ 0.98045337]]
[[ 0.06080761]]
[[ 0.22928509]]
[[ 0.97302419]]
[[ 0.01318093]]
[[ 0.00010067]]
[[ 0.87892455]]
[[ 0.9999994]]
[[ 0.99784315]]
[[ 0.99931812]]
[[ 0.82177544]]
[[ 0.17663963]]


In [17]:
ins_closedEyesModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1, 30, 30)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 1, 36, 36)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 32, 32, 32)        832       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 32, 32, 32)        128       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 32, 16, 16)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 12, 12)        51264     
__________

In [32]:
ins_closedEyesModel.save('my_model.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
import os
import cv2
import numpy as np 

from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.models import load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model

import keras.backend as K
K.set_image_data_format('channels_first')

ins_closedEyesModel = load_model('my_model.h5')

In [ ]:
ins_closedEyesModel = load_model('my_model.h5')

In [ ]:
img_arr = np.empty([1, 1, IMG_WIDTH, IMG_HEIGHT], np.uint8)
image = cv2.imread('zuc.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
img_arr[0,0] = cv2.resize(image, (IMG_WIDTH, IMG_HEIGHT), interpolation = cv2.INTER_AREA)
print(ins_closedEyesModel.predict(img_arr))
cv2.imshow('Img', img_arr[0,0])
cv2.waitKey()
cv2.destroyAllWindows()

In [30]:
def sketch(image):
    img_arr = np.empty([1, 1, 30, 30], np.uint8)
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_arr[0,0] = extract_eye(img_gray)
    pred = ins_closedEyesModel.predict(img_arr)
    #cv2.putText(img_arr[0,0], str(pred), (0,50), cv2.FONT_HERSHEY_COMPLEX, 1, (100,170,0), 2)
    if pred < 0.5 :
        cv2.putText(image, 'Open', (0,50), cv2.FONT_HERSHEY_COMPLEX, 2, (100,170,0), 3)
    else :
        cv2.putText(image, 'Close', (0,50), cv2.FONT_HERSHEY_COMPLEX, 2, (100,170,0), 3)
        
    return image

In [31]:
# Initialize webcam, cap is the object provided by VideoCapture
# It contains a boolean indicating if it was sucessful (ret)
# It also contains the images collected from the webcam (frame)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    cv2.imshow('Our Live Sketcher', sketch(frame))
    #cv2.imshow('Our Live Sketcher', frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()      

In [ ]:
# Store images from VideoCapture
import cv2

cap = cv2.VideoCapture(0)
i = 0
while True:
    ret, frame = cap.read()
    cv2.imshow('Our Live Sketcher', frame)
    cv2.imwrite('My_data\\output'+str(i)+'.jpg', frame)
    i += 1
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

# Release camera and close windows
cap.release()
cv2.destroyAllWindows()   

In [ ]:
for dirname, dirnames, filenames in os.walk('.\My_data'):

    for filename in filenames:
        image = cv2.imread('.\My_data\\'+filename)
        cv2.imshow('Check Result', sketch(image))
        cv2.waitKey()

# Close windows
cv2.destroyAllWindows()